In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv1D,Conv2D,MaxPooling2D,Flatten,AveragePooling2D,Input,BatchNormalization,AveragePooling1D
from keras.optimizers import RMSprop

2022-07-05 22:01:19.043342: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guisoares/catkin_ws/devel/lib:/opt/ros/noetic/lib:/usr/lib/x86_64-linux-gnu/:/home/guisoares/catkin_ws/install/lib/mavlink_sitl_gazebo/plugins
2022-07-05 22:01:19.043405: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Model creation
model = Sequential()
# model.add(Input(shape=(23,25600,1)))

model.add(Conv1D(4, 5, activation='relu', input_shape=(25600, 23), name='conv1'))
model.add(BatchNormalization(name='batch1'))
model.add(AveragePooling1D(pool_size=2, strides=2, name='pool1'))
# output ~(12800,4)

model.add(Conv1D(4, 6, activation='relu', name='conv2'))
model.add(BatchNormalization(name='batch2'))
model.add(AveragePooling1D(pool_size=4, strides=1,name='pool2'))
# output ~(12800,4)

model.add(Conv1D(10, 4, activation='relu', name='conv3'))
model.add(BatchNormalization(name='batch3'))
model.add(AveragePooling1D(pool_size=2, strides=2, name='pool3'))
# output ~(6400,4)

model.add(Conv1D(10, 4, activation='relu', name='conv4'))
model.add(BatchNormalization(name='batch4'))
model.add(AveragePooling1D(pool_size=2, strides=4, name='pool4'))
# output ~(1200,4)

model.add(Conv1D(15, 2, activation='relu', name='conv5'))
model.add(BatchNormalization(name='batch5'))
model.add(AveragePooling1D(pool_size=2, strides=4, name='pool5'))

model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

2022-07-05 22:01:24.287752: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guisoares/catkin_ws/devel/lib:/opt/ros/noetic/lib:/usr/lib/x86_64-linux-gnu/:/home/guisoares/catkin_ws/install/lib/mavlink_sitl_gazebo/plugins
2022-07-05 22:01:24.287799: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-05 22:01:24.287828: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (guisoares-Aspire-A315-53): /proc/driver/nvidia/version does not exist
2022-07-05 22:01:24.288152: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv1D)              (None, 25596, 4)          464       
                                                                 
 batch1 (BatchNormalization)  (None, 25596, 4)         16        
                                                                 
 pool1 (AveragePooling1D)    (None, 12798, 4)          0         
                                                                 
 conv2 (Conv1D)              (None, 12793, 4)          100       
                                                                 
 batch2 (BatchNormalization)  (None, 12793, 4)         16        
                                                                 
 pool2 (AveragePooling1D)    (None, 12790, 4)          0         
                                                                 
 conv3 (Conv1D)              (None, 12787, 10)         1

In [4]:

import keras.backend as K
def compute_binary_specificity(y_pred, y_true):
    """Compute the confusion matrix for a set of predictions.

    Parameters
    ----------
    y_pred   : predicted values for a batch if samples (must be binary: 0 or 1)
    y_true   : correct values for the set of samples used (must be binary: 0 or 1)

    Returns
    -------
    out : the specificity
    """

    # check_binary(K.eval(y_true))    # must check that input values are 0 or 1
    # check_binary(K.eval(y_pred))    # 

    TN = np.logical_and(K.eval(y_true) == 0, K.eval(y_pred) == 0)
    FP = np.logical_and(K.eval(y_true) == 0, K.eval(y_pred) == 1)

    # as Keras Tensors
    TN = K.sum(K.variable(TN))
    FP = K.sum(K.variable(FP))

    specificity = TN / (TN + FP + K.epsilon())
    return specificity

def specificity_loss_wrapper():
    """A wrapper to create and return a function which computes the specificity loss, as (1 - specificity)

    """
    # Define the function for your loss
    def specificity_loss(y_true, y_pred):
        return 1.0 - compute_binary_specificity(y_true, y_pred)

    return specificity_loss    # we return this function object

In [5]:
opt = keras.optimizers.RMSprop(learning_rate=0.00001)
# Create the loss function object using the wrapper function above
# spec_loss = specificity_loss_wrapper()
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=opt, metrics=['accuracy', tf.keras.metrics.SensitivityAtSpecificity(0.5), keras.metrics.SpecificityAtSensitivity(0.5)])

In [6]:
y = np.load('signals/chb01/labels.npy')
x = np.load('signals/chb01/signals.npy') 
x = np.swapaxes(x, 1, 2) #swap axis: N,23,25600 -> N,25600, 23
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, shuffle=True)

In [7]:
x.shape

(303, 25600, 23)

In [8]:
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

unique, counts = np.unique(y_test, return_counts=True)
print(dict(zip(unique, counts)))

{0: 206, 1: 6}
{0: 86, 1: 5}


In [9]:
callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", min_delta=0.1, patience=1)

In [10]:
model.fit(x_train, y_train, batch_size=4, epochs=10, validation_data=(x_test,y_test), callbacks=[callback])

Epoch 1/10


/home/guisoares/.virtualenvs/full/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


53/53 [==============================] - 15s 208ms/step - loss: 1.3019 - accuracy: 0.1698 - sensitivity_at_specificity: 0.5000 - specificity_at_sensitivity: 0.6602 - val_loss: 0.6101 - val_accuracy: 0.9451 - val_sensitivity_at_specificity: 0.0000e+00 - val_specificity_at_sensitivity: 0.0000e+00
Epoch 2/10
53/53 [==============================] - 8s 154ms/step - loss: 0.7467 - accuracy: 0.5708 - sensitivity_at_specificity: 1.0000 - specificity_at_sensitivity: 0.9563 - val_loss: 0.3155 - val_accuracy: 0.9451 - val_sensitivity_at_specificity: 0.0000e+00 - val_specificity_at_sensitivity: 0.0000e+00
Epoch 3/10
53/53 [==============================] - 8s 143ms/step - loss: 0.4738 - accuracy: 0.8160 - sensitivity_at_specificity: 1.0000 - specificity_at_sensitivity: 0.9951 - val_loss: 0.2190 - val_accuracy: 0.9451 - val_sensitivity_at_specificity: 0.0000e+00 - val_specificity_at_sensitivity: 0.0000e+00
Epoch 4/10
53/53 [==============================] - 8s 145ms/step - loss: 0.4303 - accuracy:

In [13]:
y_pred = model.predict(x_train[:])
y_pred = np.where(y_pred < 0.5, 0, 1)
unique, counts = np.unique(y_pred, return_counts=True)
print(dict(zip(unique, counts)))

unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))

7/7 [==============================] - 3s 346ms/step
{0: 212}
{0: 206, 1: 6}
